In [68]:
# Early Prediction of Diabetic Complications - eICU Starter Notebook

from pyspark.sql import SparkSession
from pyspark.sql.functions import lower, col
from pyspark.sql.functions import col, lit


In [45]:

# Create Spark session
spark = SparkSession.builder \
    .appName("eICU EDA") \
    .getOrCreate()

# Example path to your CSVs — adjust as needed
data_path = "Dataset/eicu-collaborative-research-database-2.0/"  # Change this to your actual folder




25/06/26 20:51:38 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [46]:
# Load patient and diagnosis tables
patient_df = spark.read.csv(f"{data_path}/patient.csv.gz", header=True, inferSchema=True)
diagnosis_df = spark.read.csv(f"{data_path}/diagnosis.csv.gz", header=True, inferSchema=True)

# Preview
patient_df.show(5)
diagnosis_df.show(5)


+-----------------+-------------------------+------+---+---------+----------+------+--------------------+---------------+-------------------+-------------------+--------------------+---------------------+-----------------------+-----------------------+-------------------------+-----------------------+------------+-------------------+--------------------+---------------+--------------+---------------+---------------+-------------------+-------------------+---------------------+-------------------+---------+
|patientunitstayid|patienthealthsystemstayid|gender|age|ethnicity|hospitalid|wardid|   apacheadmissiondx|admissionheight|hospitaladmittime24|hospitaladmitoffset| hospitaladmitsource|hospitaldischargeyear|hospitaldischargetime24|hospitaldischargeoffset|hospitaldischargelocation|hospitaldischargestatus|    unittype|    unitadmittime24|     unitadmitsource|unitvisitnumber|  unitstaytype|admissionweight|dischargeweight|unitdischargetime24|unitdischargeoffset|unitdischargelocation|unitdis

In [48]:

# Filter diagnoses with 'diabetes' in the string
diabetic_df = diagnosis_df.filter(lower(col("diagnosisstring")).like("%diabet%")) \
                          .select("patientunitstayid").distinct()

# Count diabetic patients
print("Total diabetic patients:", diabetic_df.count())


Total diabetic patients: 19935


In [49]:
# Join to patient table for demographic context
diabetic_patients = diabetic_df.join(patient_df, on="patientunitstayid", how="left")
diabetic_patients.show(5)


+-----------------+-------------------------+------+---+----------------+----------+------+--------------------+---------------+-------------------+-------------------+--------------------+---------------------+-----------------------+-----------------------+-------------------------+-----------------------+------------+-------------------+--------------------+---------------+------------+---------------+---------------+-------------------+-------------------+---------------------+-------------------+---------+
|patientunitstayid|patienthealthsystemstayid|gender|age|       ethnicity|hospitalid|wardid|   apacheadmissiondx|admissionheight|hospitaladmittime24|hospitaladmitoffset| hospitaladmitsource|hospitaldischargeyear|hospitaldischargetime24|hospitaldischargeoffset|hospitaldischargelocation|hospitaldischargestatus|    unittype|    unitadmittime24|     unitadmitsource|unitvisitnumber|unitstaytype|admissionweight|dischargeweight|unitdischargetime24|unitdischargeoffset|unitdischargelocati

In [52]:
complication_terms = {
    "comp_nephropathy": "nephropathy|renal failure|chronic kidney",
    "comp_retinopathy": "retinopathy",
    "comp_neuropathy": "neuropathy",
    "comp_cardio": "cardiovascular|heart failure|myocardial|infarction|angina",
    "comp_foot_ulcer": "foot ulcer"
}

# Add complication flags
for name, pattern in complication_terms.items():
    diagnosis_df = diagnosis_df.withColumn(
        name,
        (lower(col("diagnosisstring")).rlike(pattern)).cast("int")
    )


In [53]:
from pyspark.sql.functions import max as spark_max

# Group by patientunitstayid and get max (presence) of each complication
comp_flags = diagnosis_df.groupBy("patientunitstayid").agg(
    *[spark_max(col(flag)).alias(flag) for flag in complication_terms.keys()]
)


In [54]:
# Final joined dataframe
final_df = diabetic_patients.join(comp_flags, on="patientunitstayid", how="left").fillna(0)

# Show a few rows
final_df.select("patientunitstayid", "gender", "age", *complication_terms.keys()).show(10)


+-----------------+------+---+----------------+----------------+---------------+-----------+---------------+
|patientunitstayid|gender|age|comp_nephropathy|comp_retinopathy|comp_neuropathy|comp_cardio|comp_foot_ulcer|
+-----------------+------+---+----------------+----------------+---------------+-----------+---------------+
|           145123|  Male| 83|               0|               0|              0|          0|              0|
|           157793|  Male| 35|               0|               0|              0|          0|              0|
|           158803|  Male| 52|               0|               0|              0|          1|              0|
|           164603|  Male| 62|               0|               0|              0|          1|              0|
|           175925|Female| 51|               0|               0|              0|          1|              0|
|           185795|  Male| 68|               0|               0|              0|          1|              0|
|           188834|

In [55]:
final_df.select(["comp_nephropathy", "comp_retinopathy", "comp_neuropathy", "comp_cardio", "comp_foot_ulcer"]) \
        .groupBy().sum().show()


+---------------------+---------------------+--------------------+----------------+--------------------+
|sum(comp_nephropathy)|sum(comp_retinopathy)|sum(comp_neuropathy)|sum(comp_cardio)|sum(comp_foot_ulcer)|
+---------------------+---------------------+--------------------+----------------+--------------------+
|                 5667|                    0|                 343|           16282|                   0|
+---------------------+---------------------+--------------------+----------------+--------------------+



In [56]:
from pyspark.sql.functions import col, when

final_df = final_df.withColumn(
    "has_complication",
    when((col("comp_cardio") == 1) | (col("comp_nephropathy") == 1), 1).otherwise(0)
)


In [57]:
final_df.show()

+-----------------+-------------------------+------+---+----------------+----------+------+--------------------+---------------+-------------------+-------------------+--------------------+---------------------+-----------------------+-----------------------+-------------------------+-----------------------+------------+-------------------+--------------------+---------------+------------+---------------+---------------+-------------------+-------------------+---------------------+-------------------+---------+----------------+----------------+---------------+-----------+---------------+----------------+
|patientunitstayid|patienthealthsystemstayid|gender|age|       ethnicity|hospitalid|wardid|   apacheadmissiondx|admissionheight|hospitaladmittime24|hospitaladmitoffset| hospitaladmitsource|hospitaldischargeyear|hospitaldischargetime24|hospitaldischargeoffset|hospitaldischargelocation|hospitaldischargestatus|    unittype|    unitadmittime24|     unitadmitsource|unitvisitnumber|unitstayt

In [59]:
# Load lab.csv.gz
lab_df = spark.read.csv( data_path + "lab.csv.gz", header=True, inferSchema=True )

# Filter for diabetic patients
lab_df = lab_df.join(diabetic_df.select("patientunitstayid"), on="patientunitstayid", how="inner")

# Show common labs
lab_df.groupBy("labname").count().orderBy("count", ascending=False).show(20, truncate=False)


+----------------+------+
|labname         |count |
+----------------+------+
|bedside glucose |694363|
|potassium       |190019|
|sodium          |175324|
|glucose         |166634|
|chloride        |164156|
|creatinine      |163933|
|BUN             |162684|
|bicarbonate     |159027|
|calcium         |158886|
|Hgb             |154453|
|Hct             |153595|
|platelets x 1000|139689|
|WBC x 1000      |137833|
|RBC             |137350|
|MCV             |136620|
|MCHC            |136498|
|anion gap       |131888|
|RDW             |131454|
|MCH             |128952|
|magnesium       |83693 |
+----------------+------+
only showing top 20 rows



In [60]:
from pyspark.sql.functions import col, lower

# Focused lab names
target_labs = ["creatinine", "glucose", "bedside glucose"]

# Filter for diabetic patients' lab tests for creatinine and glucose
filtered_labs = (
    lab_df
    .filter(lower(col("labname")).isin([name.lower() for name in target_labs]))
    .join(diabetic_df.select("patientunitstayid"), on="patientunitstayid", how="inner")
)

filtered_labs.show(10, truncate=False)


+-----------------+--------+---------------+---------+---------------+---------+-------------+--------------------+-----------------------+----------------------+
|patientunitstayid|labid   |labresultoffset|labtypeid|labname        |labresult|labresulttext|labmeasurenamesystem|labmeasurenameinterface|labresultrevisedoffset|
+-----------------+--------+---------------+---------+---------------+---------+-------------+--------------------+-----------------------+----------------------+
|141203           |58323261|-25            |4        |bedside glucose|143.0    |143          |mg/dL               |mg/dL                  |-22                   |
|141203           |46463462|-473           |1        |glucose        |95.0     |95           |mg/dL               |mg/dL                  |-412                  |
|141203           |45761373|1              |1        |glucose        |145.0    |145          |mg/dL               |mg/dL                  |78                    |
|141203           |457

In [61]:
from pyspark.sql.functions import when, avg, max as spark_max, count, col

# Create separate DataFrames for each test
creatinine_df = filtered_labs.filter(lower(col("labname")) == "creatinine")
glucose_df = filtered_labs.filter(lower(col("labname")) == "glucose")
bedside_glucose_df = filtered_labs.filter(lower(col("labname")) == "bedside glucose")

# Aggregate: Creatinine
creatinine_agg = creatinine_df.groupBy("patientunitstayid").agg(
    avg("labresult").alias("creatinine_avg"),
    spark_max("labresult").alias("creatinine_max"),
    count("labresult").alias("creatinine_count")
)

# Aggregate: Glucose
glucose_agg = glucose_df.groupBy("patientunitstayid").agg(
    avg("labresult").alias("glucose_avg"),
    spark_max("labresult").alias("glucose_max"),
    count("labresult").alias("glucose_count")
)

# Aggregate: Bedside Glucose
bedside_glucose_agg = bedside_glucose_df.groupBy("patientunitstayid").agg(
    avg("labresult").alias("bedside_glucose_avg"),
    spark_max("labresult").alias("bedside_glucose_max"),
    count("labresult").alias("bedside_glucose_count")
)


In [62]:
# Merge all lab features
diabetic_labs_df = (
    diabetic_df
    .join(creatinine_agg, on="patientunitstayid", how="left")
    .join(glucose_agg, on="patientunitstayid", how="left")
    .join(bedside_glucose_agg, on="patientunitstayid", how="left")
    .fillna(0)  # Fill missing lab values with 0 or appropriate imputation later
)

diabetic_labs_df.show(5)


+-----------------+------------------+--------------+----------------+------------------+-----------+-------------+-------------------+-------------------+---------------------+
|patientunitstayid|    creatinine_avg|creatinine_max|creatinine_count|       glucose_avg|glucose_max|glucose_count|bedside_glucose_avg|bedside_glucose_max|bedside_glucose_count|
+-----------------+------------------+--------------+----------------+------------------+-----------+-------------+-------------------+-------------------+---------------------+
|           145123|0.7666666666666666|           0.9|               3|             213.0|      213.0|            1| 139.78947368421052|              248.0|                   19|
|           157793|1.0833333333333335|           1.7|               6|207.33333333333334|      458.0|            6|              156.0|              320.0|                   33|
|           158803| 6.866666666666667|          8.56|               6|178.66666666666666|      206.0|         

In [63]:
diabetic_labs_df.select(numeric_columns).describe().show()


NameError: name 'numeric_columns' is not defined

In [64]:
from pyspark.sql.functions import col, isnan, when, count

diabetic_labs_df.select([
    count(when(col(c).isNull() | isnan(c), c)).alias(c)
    for c in diabetic_labs_df.columns
]).show()


+-----------------+--------------+--------------+----------------+-----------+-----------+-------------+-------------------+-------------------+---------------------+
|patientunitstayid|creatinine_avg|creatinine_max|creatinine_count|glucose_avg|glucose_max|glucose_count|bedside_glucose_avg|bedside_glucose_max|bedside_glucose_count|
+-----------------+--------------+--------------+----------------+-----------+-----------+-------------+-------------------+-------------------+---------------------+
|                0|             0|             0|               0|          0|          0|            0|                  0|                  0|                    0|
+-----------------+--------------+--------------+----------------+-----------+-----------+-------------+-------------------+-------------------+---------------------+



In [66]:
from pyspark.sql.functions import col, min as spark_min

# Ensure 'diagnosis_df' and 'diabetic_df' already exist and include complication flags
# You must have columns like 'comp_nephropathy', etc. in diagnosis_df already

# 1. Create a diagnosis offset in days
diagnosis_df = diagnosis_df.withColumn(
    "diagnosis_offset_days", (col("diagnosisoffset").cast("int") / 1440).cast("int")
)

# 2. Diabetes onset per patient
diabetes_onset = diagnosis_df.filter(col("diagnosisstring").rlike("(?i)diabetes")).groupBy("patientunitstayid").agg(
    spark_min("diagnosis_offset_days").alias("first_diabetes_day")
)

# 3. Complication onsets
complication_cols = ["comp_nephropathy", "comp_retinopathy", "comp_neuropathy", "comp_cardio", "comp_foot_ulcer"]

complication_dates = []
for comp in complication_cols:
    comp_df = diagnosis_df.filter(col(comp) == 1).groupBy("patientunitstayid").agg(
        spark_min("diagnosis_offset_days").alias(f"first_{comp}_day")
    )
    complication_dates.append(comp_df)

# 4. Merge all into a single timeline DataFrame
timeline_df = diabetic_df.select("patientunitstayid").distinct().join(diabetes_onset, "patientunitstayid", "left")
for comp_df in complication_dates:
    timeline_df = timeline_df.join(comp_df, "patientunitstayid", "left")

# 5. Show the timeline for manual review
timeline_df.orderBy("first_diabetes_day").show(10)


+-----------------+------------------+--------------------------+--------------------------+-------------------------+---------------------+-------------------------+
|patientunitstayid|first_diabetes_day|first_comp_nephropathy_day|first_comp_retinopathy_day|first_comp_neuropathy_day|first_comp_cardio_day|first_comp_foot_ulcer_day|
+-----------------+------------------+--------------------------+--------------------------+-------------------------+---------------------+-------------------------+
|          3140789|              NULL|                      NULL|                      NULL|                     NULL|                 NULL|                     NULL|
|          1793086|              NULL|                      NULL|                      NULL|                     NULL|                    0|                     NULL|
|          1123518|              NULL|                      NULL|                      NULL|                     NULL|                    0|                     NULL

In [69]:
# Optional: Flag diabetics based on high glucose or insulin meds (if available)
high_glucose_df = lab_df.filter(
    (col("labname") == "glucose") & (col("labresult").cast("float") > 200)
).select("patientunitstayid").distinct().withColumn("likely_diabetic", lit(1))

# Join this with diabetic_df to identify additional suspected cases


In [70]:
high_glucose_df.show(10)

+-----------------+---------------+
|patientunitstayid|likely_diabetic|
+-----------------+---------------+
|           145123|              1|
|           157793|              1|
|           158803|              1|
|           164603|              1|
|           234179|              1|
|           239049|              1|
|           276652|              1|
|           323084|              1|
|           334772|              1|
|           351086|              1|
+-----------------+---------------+
only showing top 10 rows



In [71]:
# Merge with original diabetic_df (left join to keep all known diabetics)
expanded_diabetic_df = diabetic_df.join(
    high_glucose_df, on="patientunitstayid", how="outer"
).fillna({"likely_diabetic": 0})

expanded_diabetic_df.show(5)


+-----------------+---------------+
|patientunitstayid|likely_diabetic|
+-----------------+---------------+
|           141203|              0|
|           141266|              1|
|           141328|              0|
|           141392|              0|
|           141462|              1|
+-----------------+---------------+
only showing top 5 rows



In [72]:
full_diabetic_df = diabetic_df.join(
    high_glucose_df, on="patientunitstayid", how="outer"
).fillna({"likely_diabetic": 0})


In [74]:
# 1. Merge complication flags with full_diabetic_df
enriched_diabetic_df = full_diabetic_df.join(
    complication_flags,
    on="patientunitstayid",
    how="left"
).fillna(0)

# 2. Add has_complication column using greatest()
from pyspark.sql.functions import greatest, col

enriched_diabetic_df = enriched_diabetic_df.withColumn(
    "has_complication",
    greatest(
        col("comp_nephropathy"),
        col("comp_retinopathy"),
        col("comp_neuropathy"),
        col("comp_cardio"),
        col("comp_foot_ulcer")
    )
)


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `comp_nephropathy` cannot be resolved. Did you mean one of the following? [`nephropathy`, `neuropathy`, `retinopathy`, `foot_ulcer`, `cardiovascular`].;
'Project [patientunitstayid#5463, likely_diabetic#5464, nephropathy#5465, retinopathy#5466, neuropathy#5467, cardiovascular#5468, foot_ulcer#5469, greatest('comp_nephropathy, 'comp_retinopathy, 'comp_neuropathy, 'comp_cardio, 'comp_foot_ulcer) AS has_complication#5477]
+- Project [coalesce(patientunitstayid#5440, cast(0.0 as int)) AS patientunitstayid#5463, coalesce(likely_diabetic#5445, cast(0.0 as int)) AS likely_diabetic#5464, coalesce(nephropathy#3090, cast(0.0 as int)) AS nephropathy#5465, coalesce(retinopathy#3092, cast(0.0 as int)) AS retinopathy#5466, coalesce(neuropathy#3094, cast(0.0 as int)) AS neuropathy#5467, coalesce(cardiovascular#3096, cast(0.0 as int)) AS cardiovascular#5468, coalesce(foot_ulcer#3098, cast(0.0 as int)) AS fo